In [11]:
import pandas as pd

from scipy.stats import chisquare
from scipy.stats import chi2_contingency

In [18]:
# Import the original dataset

df = pd.read_csv("data/sbl_grouped_cmpls.csv").fillna("")

In [19]:
pd.set_option("display.max_columns", None)

In [20]:
df

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse,construction
0,212418,BW>[,MT,Isaiah,1,23,JBW>,FRJK SWRRJM W XBRJ GNBJM KLW >HB CXD W RDP CLM...,,qal,impf,W RJB >LMNH L> JBW> >LJHM,RJB >LMNH,>LJHM,>L,to them,0.0,prep,prps,anim,det,prsf,simple,fictive,>L,,,,,,prep
1,212437,CWB[,MT,Isaiah,1,25,>CJBH,W >CJBH JDJ <LJK W >YRP K BR SJGJK W >SJRH KL ...,,hif,impf,W >CJBH JDJ <LJK,,<LJK,<L,against you,0.0,prep,prps,anim,det,prsf,simple,factive fictive,<L,,,,,,prep
2,212564,NHR[,MT,Isaiah,2,2,NHRW,W HJH B >XRJT H JMJM NKWN JHJH HR BJT JHWH B R...,,qal,perf,W NHRW >LJW KL H GWJM,KL H GWJM,>LJW,>L,to it (the mountain),0.0,prep,prps,inanim,det,prsf,simple,factive,>L,,,,,,prep
3,212577,<LH[,MT,Isaiah,2,3,N<LH,W HLKW <MJM RBJM W >MRW LKW W N<LH >L HR JHWH ...,,qal,impf,W N<LH >L HR JHWH >L BJT >LHJ J<QB,,>L HR JHWH >L BJT >LHJ J<QB,>L HR/ JHWH/ >L BJT/ >LHJM/ J<QB/,to the mountain of Yahwe to the house of the G...,0.0,prep,topo,inanim,det,subs,complex,factive,>L,>L,,,,,prep
4,212590,HLK[,MT,Isaiah,2,3,NLKH,W HLKW <MJM RBJM W >MRW LKW W N<LH >L HR JHWH ...,,qal,impf,W NLKH B >RXTJW,,B >RXTJW,B >RX/,in his paths,0.0,prep,abs,inanim,det,subs,complex,factive fictive,B,,,,,,prep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,33027,BW>[,MT,Exodus,9,1,B>,W J>MR JHWH >L MCH B> >L PR<H W DBRT >LJW KH >...,,qal,impv,B> >L PR<H,,>L PR<H,>L PR<H/,to Pharaoh,0.0,prep,pers,anim,det,subs,simple,factive,>L,,,,,,prep
655,33436,JRD[,MT,Exodus,9,19,JRD,W <TH CLX H<Z >T MQNK W >T KL >CR LK B FDH KL ...,,qal,perf,W JRD <LHM H BRD,H BRD,<LHM,<L,on them (all that is found in the field),0.0,prep,prps,inanim,det,prsf,simple,factive fictive,<L,,,,"the hail goes down on them, vertical motion",,prep
656,33450,NWS[,MT,Exodus,9,20,HNJS,H JR> >T DBR JHWH M <BDJ PR<H HNJS >T <BDJW W ...,,hif,perf,HNJS >T <BDJW W >T MQNHW >L H BTJM,,>L H BTJM,>L H BJT/,into the houses,0.0,prep,topo,inanim,det,subs,simple,factive,>L,,,,,,prep
657,33526,HLK[,MT,Exodus,9,23,THLK,W JV MCH >T MVHW <L H CMJM W JHWH NTN QLT W BR...,,qal,wayq,W THLK >C >RYH,>C,>RYH,>RY/,toward the earth,1.0,dir-he,topo,inanim,det,subs,simple,factive fictive,,,,,"vertical motion? as the fire comes from God, l...",,non-prep


In [21]:
# Define q Chi Square Test of Independence Function

def perform_chi_square_test(data, col1, col2):
    # Creating a contingency table
    contingency_table = pd.crosstab(data[col1], data[col2])

    # Performing the Chi-Square Test
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Interpreting the result
    significant = p < 0.05  # 5% significance level
    return chi2, p, significant

In [22]:
cont_table = pd.crosstab(df["construction"], df["cmpl_anim"])
cont_table

cmpl_anim,anim,inanim
construction,,
non-prep,1,120
prep,198,340


In [25]:
len(df)

659

In [23]:
df[(df.construction == "non-prep") & (df.cmpl_anim == "anim")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse,construction
33,216346,<WP[,MT,Isaiah,11,14,<PW,W <PW B KTP PLCTJM JMH JXDW JBZW >T BNJ QDM >D...,,qal,perf,W <PW B KTP PLCTJM JMH,,PLCTJM,PLCTJ/,towards the Philistines,0.0,vc,gens,anim,und,nmpr,simple,factive,,,,,,,non-prep


In [24]:
perform_chi_square_test(df, "construction", "cmpl_constr")

(659.0, 4.337955781787197e-139, True)

In [27]:
cont_table = pd.crosstab(df["construction"], df["cmpl_det"])
cont_table

cmpl_det,det,und
construction,,
non-prep,110,11
prep,512,26


In [26]:
perform_chi_square_test(df, "construction", "cmpl_det")

(2.624187150649214, 0.10524638986744972, False)